In [58]:
import pandas as pd
import os
import sqlite3
import numpy as np

import geopandas as gpd
from lets_plot.geo_data import *
from numerize import numerize as nz

from lets_plot import * # This imports all of ggplot2's functions
LetsPlot.setup_html() 

In [59]:
DATA_FOLDER_P = os.path.join('../data')
conn = sqlite3.connect(os.path.join(DATA_FOLDER_P, './home_prices.db'))


In [60]:
zip_df = pd.read_sql_query("SELECT * FROM zip_info", conn)
zillow_df = pd.read_sql_query("SELECT * FROM zillow", conn)
display(zillow_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              911 non-null    int64  
 1   price           911 non-null    float64
 2   address         911 non-null    object 
 3   addressZipcode  911 non-null    object 
 4   addressState    911 non-null    object 
 5   beds            911 non-null    int64  
 6   baths           911 non-null    float64
 7   area            911 non-null    float64
 8   zestimate       674 non-null    float64
 9   brokerName      815 non-null    object 
 10  town            911 non-null    object 
dtypes: float64(4), int64(2), object(5)
memory usage: 78.4+ KB


None

In [66]:
zillow_df[zillow_df['addressZipcode'] == '2118'] = zillow_df[zillow_df['addressZipcode'] == '2118'].replace('2118', '02118')

In [67]:
agg_funcs = {
    'price': ['mean', 'median', 
                 ('25th', lambda x: np.percentile(x, 25)),  
                 ('75th', lambda x: np.percentile(x, 75)),  
                 'min', 'max', 'count']
}
#using code from midterm
price_zip = zillow_df.groupby('addressZipcode').agg(agg_funcs)
#combining the two header lines onto one line
price_zip.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in price_zip.columns]
price_zip = price_zip.reset_index()
display(price_zip)

,addressZipcode,price_mean,price_median,price_25th,price_75th,price_min,price_max,price_count
0,02108,6.102589e+06,4200000.0,2549250.0,9550000.0,639000.0,17995000.0,28
1,02109,1.474444e+06,1100000.0,895000.0,2395000.0,740000.0,2695000.0,9
2,02110,2.981574e+06,2232500.0,1263500.0,3799250.0,625000.0,13955000.0,80
3,02111,2.064845e+06,1249750.0,949250.0,1993750.0,579000.0,13450000.0,50
4,02113,2.245406e+06,1462000.0,782875.0,2362500.0,399999.0,9400000.0,16
5,02114,2.086753e+06,1307500.0,776000.0,2950000.0,499500.0,7950000.0,58
6,02115,6.147028e+06,3750000.0,1099250.0,7837500.0,459000.0,35000000.0,50
7,02116,3.854766e+06,2372500.0,1260500.0,4490000.0,475000.0,27000000.0,130
8,02118,1.586557e+06,1100000.0,835000.0,1699000.0,479000.0,11500000.0,93
9,02127,1.279949e+06,999900.0,782000.0,1575000.0,419900.0,5000000.0,135


In [69]:
p = (ggplot(price_zip) + 
    geom_boxplot(aes(x='addressZipcode', ymin='price_min', lower='price_25th', middle='price_median', upper='price_75th', ymax='price_max'), stat='identity') +\
    scale_y_log10() )
    

p.show()

In [70]:
GEO_JSON_PATH = '../data/ZIP_Codes.geojson'
gdf = gpd.read_file(GEO_JSON_PATH)
display(gdf)
# gdf = gpd.read_file(shapefile_path)